# Habitica API Test Page
This page is to test queries against the Habitica API. I am primarily using it to inspect the structure of JSON data objects to help with mapping to C# classes.

To avoid authentication details being exposed to the world in this file, it requires the creation of a `.auth.cfg` (any name really, but that is the default) with the following content:

    [Habitica]
    url = https://habitica.com
    login = ;your user ID
    password = ;your API key

Note that the section name can be anything, and you can use multiple sections for multiple accounts.

# Setup

In [ ]:
import os
import requests
from configparser import ConfigParser

In [ ]:
class ConfigError(Exception):
    def __init__(self, value):
        self.value = value
        
    def __str__(self):
        return repr(self.value)

In [ ]:
def load_auth(config_file='~/.auth.cfg', section='Habitica'):
    headers = {}
    config_file = os.path.expanduser(config_file)
    
    with open(config_file) as cf:
        config = ConfigParser()
        config.read_file(cf)

        try:
            headers = {'url': config.get(section, 'url'),
                  'x-api-user': config.get(section, 'login'),
                  'x-api-key': config.get(section, 'password')}
        except configparser.NoSectionError:
            raise ConfigError("No '%s' section in '%s'" % section, configfile)
        except configparser.NoOptionError as e:
            raise ConfigError("Missing option in auth file '%s': %s" % (configfile, e.message))

    return headers

In [ ]:
headers = load_auth(section='Habitica_testuser')
#headers = load_auth(section='Habitica_deecee')
headers['url'] += '/api/v3/'

# HTTP Request Helpers

In [ ]:
def get(command):
    r = requests.get(headers['url']+command, headers=headers)
    return r.json()

# Status

In [ ]:
get('status')

# User

In [ ]:
user = get('user')['data']

In [ ]:
user.keys()

In [ ]:
user['history']

In [ ]:
user['tags']

# Member

In [ ]:
member = get('members/'+headers['x-api-user'])['data']

In [ ]:
member.keys()

In [ ]:
member

# Task

### Find the common set of keys to all task types

In [ ]:
all_tasks = get('tasks/user')['data']

In [ ]:
habits = [task for task in all_tasks if task['type'] == 'habit']
dailies = [task for task in all_tasks if task['type'] == 'daily']
todos = [task for task in all_tasks if task['type'] == 'todo']
rewards = [task for task in all_tasks if task['type'] == 'reward']

In [ ]:
with_alias = [t['type'] for t in all_tasks if 'alias' in t]
with_alias

In [ ]:
tasks = [
    set(habits[0]),
    set(rewards[0]), 
    set(todos[0]), 
    set(dailies[0])]

base_task = tasks[0]
for t in tasks[1:]:
    base_task = base_task.intersection(t)
    
base_task

In [ ]:
# all keys in habit that are not in base_task
tasks[0].difference(base_task)

In [ ]:
# all keys in reward that are not in base_task
tasks[1].difference(base_task)

In [ ]:
# all keys in todo that are not in base_task
tasks[2].difference(base_task)

In [ ]:
# all keys in daily that are not in base_task
tasks[3].difference(base_task)

### Examine components of a task in detail

In [ ]:
dailies[1]

# Groups

In [ ]:
groupId = 'habitrpg'
g = get('groups/' + groupId)
g